In [ ]:
# Not needed if tinc-python is installed
import sys
sys.path.append('../external/tinc/tinc-python/tinc-python')

In [2]:
from tinc_client import *
tclient = TincClient()

Attempt connection. localhost:34450


In [3]:
tclient.synchronize()

Connected, sending handshake.
Got HANDSHAKE_ACK. Server version 0 revision 0
ParameterSpace already registered: 'tinc::IdObject@2327163668504'


In [4]:
tclient.print()

Print
TINC Server
CONNECTED
 ** Parameter ShellSiteTypes group:  (<class 'int'>)
    Default: 0
    Min: 0
    Max: 127
 ** Parameter markerColor group:  (<class 'list'>)
    Default: []
    Min: [0, 0, 0, 0]
    Max: [1, 1, 1, 1]
 ** Parameter markerScale group:  (<class 'float'>)
    Default: 0.0
    Min: 0.0010000000474974513
    Max: 20.0
 ** Parameter dataset group:  (<class 'str'>)
    Default: 
 ** ParameterSpace tinc::IdObject@2327163668504
 ** DiskBuffer: 'graph' type 0
      path: C:\Users\Andres\source\repos\casm_viewer\bin\cachedGraph\ basename: currentGraph.png
 ** DataPool: resultsData
      ParameterSpace id: tinc::IdObject@2327163668504
ParameterSpace already registered: 'tinc::IdObject@2327163668504'
ParameterSpace already registered: 'tinc::IdObject@2327163668504'
ParameterSpace already registered: 'tinc::IdObject@2327163668504'
ParameterSpace already registered: 'tinc::IdObject@2327163668504'
ParameterSpace already registered: 'tinc::IdObject@2327163668504'
Parameter

The 'graph' ```imageBuffer``` determines what graph is shown.

In [5]:
tclient.parameter_spaces[0].get_current_path()

'33\\conditions.0\\'

In [6]:
tclient.parameter_spaces[0].get_root_path()

'C:\\Users\\Andres\\source\\repos\\vdv_data\\MgTiS_spinel_diffusion_visualization\\event_tracker_testing_quick\\'

Connection closed.
Attempt connection. localhost:34450
Connection failed.
Closed command server


In [7]:
imageBuffer = tclient.get_disk_buffer('graph')

In [8]:
imageBuffer._path

'C:\\Users\\Andres\\source\\repos\\casm_viewer\\bin\\cachedGraph\\'

In [9]:
import random
import matplotlib.pyplot as plt
import threading

def update_graph(parameter_value):
    print("Parameter value " + str(parameter_value))
    data = [random.random() * parameter_value for i in range(10)]
    with threading.Lock():
        fname = imageBuffer.get_filename_for_writing()
        f = plt.figure()
        plt.title(f" Time: {parameter_value}")
        plt.plot(data)
        #print("Update " + fname)
        plt.savefig(fname)
        # The following to avoid extra printout in jupyter cell
        plt.close()
        f.clf()
        del f
        imageBuffer.done_writing_file(fname)

In [11]:
update_graph(40)

Parameter value 40


In [4]:
shellsites = tclient.get_parameter("ShellSiteTypes")

In [9]:
tclient.request_parameter_choice_elements(shellsites)

['cleared_path0.nc', 'double_cleared_path0.nc', 'forced_path0.nc', 'free_range_path0.nc', 'oct_doub_va0.nc', 'oct_no_va0.nc', 'oct_sing_va0.nc', 'oct_trip_va0.nc', 'queued_path0.nc', 'semiforced_path0.nc', 'tet_no_va0.nc', 'tet_sing_va0.nc']


In [10]:
shellsites.get_current_elements()

['free_range_path0.nc']

In [ ]:
shellsites.register_callback(update_graph)

In [7]:
shellsites.value

136

In [8]:
ms = tclient.get_parameter("markerScale")

In [9]:
ms.value

1.7000000476837158

In [11]:
ms.value = 5

send


In [ ]:
mc = tclient.get_parameter("markerColor")

In [ ]:
mc.value

In [ ]:
mc.value = [0.5, 1, 0.5, 1]

In [ ]:
import matplotlib

In [ ]:
matplotlib.colors.to_rgb('blue')

In [ ]:
colors = ['blue', 'red', 'green', 'white', 'darkBlue', 'cyan', 'magenta', 'lightBlue','blue', 'red', 'green', 'cyan', 'magenta', 'lightBlue']

In [ ]:
[matplotlib.colors.to_rgb(c) for c in colors]

In [ ]:
bin(shellsites.value)

In [26]:
def set_color(shellSite):
    b = int(shellSite)
    for c in colors:
        if b & 1 == 1:
            rgb = list(matplotlib.colors.to_rgb(c))
            rgb.append(1)
            mc.value = rgb
            break
        b = b >> 1
    

In [27]:
set_color(1024)

send


In [29]:
shellsites.register_callback(set_color)

Replacing previously registered callback


In [4]:
dp = tclient.get_datapool("resultsData")

In [6]:
dp.slice_cache_dir

'C:\\Users\\Andres\\source\\repos\\vdv_data\\MgTiS_spinel_diffusion_visualization\\event_tracker_testing_quick\\slices\\'

In [7]:
dp.get_slice("<comp(a)>", "chempotA")

masked_array(data=[1.21477162e-04, 6.07385824e-04, 3.52283777e-03,
                   1.04470355e-02, 1.93148684e-02, 2.90330425e-02,
                   3.74149643e-02, 4.62827981e-02, 5.69727905e-02,
                   6.74198270e-02, 7.48299286e-02, 8.57628733e-02,
                   9.37803686e-02, 1.03862971e-01, 1.14431486e-01,
                   1.23542272e-01, 1.32288635e-01, 1.41277939e-01,
                   1.50874630e-01, 1.59620985e-01, 1.68367341e-01,
                   1.78449959e-01, 1.89139947e-01, 1.96914479e-01,
                   2.06754133e-01, 2.16958210e-01, 2.29105935e-01,
                   2.41375118e-01, 2.51093298e-01, 2.65792042e-01,
                   2.77332366e-01, 2.89480090e-01, 2.99441218e-01,
                   3.10374141e-01, 3.20456743e-01, 3.31268221e-01,
                   3.44873667e-01, 3.55199218e-01, 3.66739541e-01,
                   3.75485897e-01, 3.88241023e-01, 3.97108853e-01,
                   4.07191455e-01, 4.19703603e-01, 4.29907680e

In [12]:
dir_param = tclient.get_parameter("dir")

ParameterSpace already registered: 'tinc::IdObject@2931829647216'
ParameterSpace already registered: 'tinc::IdObject@2931829647216'
ParameterSpace already registered: 'tinc::IdObject@2931829647216'
ParameterSpace already registered: 'tinc::IdObject@2931829647216'
ParameterSpace already registered: 'tinc::IdObject@2931829647216'


In [13]:
dir_param.value

44.0

In [14]:
dir_param.value = 70

send
ParameterSpace already registered: 'tinc::IdObject@2931829647216'
ParameterSpace already registered: 'tinc::IdObject@2931829647216'
ParameterSpace already registered: 'tinc::IdObject@2931829647216'
ParameterSpace already registered: 'tinc::IdObject@2931829647216'
ParameterSpace already registered: 'tinc::IdObject@2931829647216'


In [40]:
dir_param.value = 30
dp.get_slice("<comp(c)>", "chempotA")

send
ParameterSpace already registered: 'tinc::IdObject@2335365395408'
ParameterSpace already registered: 'tinc::IdObject@2335365395408'
ParameterSpace already registered: 'tinc::IdObject@2335365395408'
ParameterSpace already registered: 'tinc::IdObject@2335365395408'
ParameterSpace already registered: 'tinc::IdObject@2335365395408'
Parameter value 136
send
Parameter value 136
send
Connection closed.
Attempt connection. localhost:34450
Connected, sending handshake.
Got HANDSHAKE_ACK. Server version 0 revision 0
ParameterSpace already registered: 'tinc::IdObject@2931829647216'
ParameterSpace already registered: 'tinc::IdObject@2931829647216'
ParameterSpace already registered: 'tinc::IdObject@2931829647216'


KeyboardInterrupt: 

In [15]:
dp

NameError: name 'dp' is not defined

In [6]:
dp.slice_cache_dir

NameError: name 'dp' is not defined

In [8]:
param=tclient.parameter_spaces[0]
root_path=param.get_root_path()
trajectories_path=root_path+param.get_current_path()+"trajectory.nc"

In [9]:
trajectories_path

'C:\\Users\\Andres\\source\\repos\\vdv_data\\MgTiS_spinel_diffusion_visualization\\event_tracker_testing_quick\\24\\conditions.0\\trajectory.nc'

In [10]:
import netCDF4
nc = netCDF4.Dataset(trajectories_path)
nc.variables['occupation_dofs']
#choose an atom to track
#find the first step that site index changes, record new position
#track that position from then on
# repeat if changes.
atom_to_track=0
steps_changed=[]
previous_dof=nc.variables['occupation_dofs'][2000].data
print(previous_dof)

[0 1 0 ... 0 0 0]


In [11]:
len(nc.variables['occupation_dofs'][2000].data)

24696

In [22]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
!ncdump -c C:\\Users\\Andres\\source\\repos\\vdv_data\\MgTiS_spinel_diffusion_visualization\\event_tracker_testing_quick\\24\\conditions.0\\trajectory.nc

netcdf trajectory {
dimensions:
	sample_dim = 3000 ;
	config_dim = 24696 ;
variables:
	int passes(sample_dim) ;
	int steps(sample_dim) ;
	ubyte occupation_dofs(sample_dim, config_dim) ;
data:
}


In [13]:
time = zeros((3000, 24696))

In [3]:
import xarray as xr

In [12]:
ds = xr.open_dataset(trajectories_path, chunks = {'sample_dim':10})

In [16]:
equals = []
for i in range(len(ds['occupation_dofs']) -1):
    equals.append(ds['occupation_dofs'][i] == ds['occupation_dofs'][i + 1])

In [24]:
%%timeit
argwhere(equals)

The slowest run took 383.14 times longer than the fastest. This could mean that an intermediate result is being cached.
12h 47min 13s ± 19h 29min 29s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [14]:
list_of_occ_dofs=[]
for time_slice in nc.variables['occupation_dofs']:
    print (".")
    list_of_occ_dofs.append(time_slice)

.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.


KeyboardInterrupt: 

In [ ]:
for step_count in range(nc.variables['occupation_dofs'].shape[0]):
    new_dof=nc.variables['occupation_dofs'][step_count].data
    if previous_dof[atom_to_track]!=new_dof[atom_to_track]:
        steps_changed.append(step_count)
        for i in len(previous_dof):
            if i!=atom_to_track and previous_dof[i]!=new_dof[i]:
                atom_to_track=i
               
    previous_dof=new_dof
   
print(steps_changed)